## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
app_drop = application_df.drop(["EIN", "NAME"], axis='columns')
app_drop.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Determine the number of unique values in each column.
app_drop.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
app_drop.APPLICATION_TYPE.value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# In the original code the cut off was at 1065. 
# Change to 725 for more features after one-hot encoding
app_types_replace = app_drop.apply(lambda x: x.mask(x.map(x.value_counts())<725, 'Other') 
                                   if x.name=='APPLICATION_TYPE' else x)
app_types_replace["APPLICATION_TYPE"].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
Other      804
T8         737
T7         725
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Original code resulted in 6 unique bins
app_types_replace["APPLICATION_TYPE"].nunique()

8

In [7]:
# Look at CLASSIFICATION value counts for binning
app_drop["CLASSIFICATION"].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [8]:
# In the original code the cut off was at 1883. 
# Change to 725 for more features after one-hot encoding
app_types_replace = app_types_replace.apply(lambda x: x.mask(x.map(x.value_counts())<725,'Other')
                                            if x.name=='CLASSIFICATION' else x)
app_types_replace["CLASSIFICATION"].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

In [9]:
# Original code resulted in 6 unique bins
app_types_replace["CLASSIFICATION"].nunique()

7

In [10]:
# Convert categorical data to numeric with `pd.get_dummies`
app_numeric = pd.get_dummies(app_types_replace)
app_numeric

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34295,1,5000,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34296,1,5000,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34297,1,5000,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0


In [11]:
# Split the preprocessed data into our features and target arrays
y = app_numeric["IS_SUCCESSFUL"]
X = app_numeric.drop("IS_SUCCESSFUL", axis=1)

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [12]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [13]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn_model = tf.keras.models.Sequential()

# First hidden layer
nn_model.add(tf.keras.layers.Dense(units=15, activation="relu", input_dim=43))

# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=25, activation="relu"))

# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 15)                660       
                                                                 
 dense_1 (Dense)             (None, 25)                400       
                                                                 
 dense_2 (Dense)             (None, 1)                 26        
                                                                 
Total params: 1,086
Trainable params: 1,086
Non-trainable params: 0
_________________________________________________________________


In [14]:
# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [15]:
# Train the model
# fit_model = nn.fit(X_train_scaled,y_train, epochs=100)

fit_model = nn_model.fit(X_train_scaled,y_train, epochs=50)


Epoch 1/50
804/804 [==============================] - 1s 740us/step - loss: 0.5971 - accuracy: 0.6984
Epoch 2/50
804/804 [==============================] - 1s 738us/step - loss: 0.5588 - accuracy: 0.7249
Epoch 3/50
804/804 [==============================] - 1s 715us/step - loss: 0.5539 - accuracy: 0.7261
Epoch 4/50
804/804 [==============================] - 1s 717us/step - loss: 0.5512 - accuracy: 0.7276
Epoch 5/50
804/804 [==============================] - 1s 674us/step - loss: 0.5501 - accuracy: 0.7290
Epoch 6/50
804/804 [==============================] - 1s 675us/step - loss: 0.5489 - accuracy: 0.7289
Epoch 7/50
804/804 [==============================] - 1s 670us/step - loss: 0.5477 - accuracy: 0.7299
Epoch 8/50
804/804 [==============================] - 1s 666us/step - loss: 0.5476 - accuracy: 0.7297
Epoch 9/50
804/804 [==============================] - 1s 667us/step - loss: 0.5468 - accuracy: 0.7303
Epoch 10/50
804/804 [==============================] - 1s 714us/step - loss: 0.546

In [16]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5541 - accuracy: 0.7268 - 224ms/epoch - 835us/step
Loss: 0.5540975332260132, Accuracy: 0.7267638444900513


In [17]:
# Attempt 1 Result: Accuracy decreased because of the increase of binning.

In [18]:
# Attempt 2: change units and add another layer.

In [19]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE

nn_model = tf.keras.models.Sequential()

# First hidden layer
# nn_model.add(tf.keras.layers.Dense(units=15, activation="relu", input_dim=40))
nn_model.add(tf.keras.layers.Dense(units=20, activation="relu", input_dim=43))

# Second hidden layer
# nn_model.add(tf.keras.layers.Dense(units=25, activation="relu"))
nn_model.add(tf.keras.layers.Dense(units=30, activation="relu"))

# Third hidden layer (New Layer)
nn_model.add(tf.keras.layers.Dense(units=35, activation="relu"))

# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 20)                880       
                                                                 
 dense_4 (Dense)             (None, 30)                630       
                                                                 
 dense_5 (Dense)             (None, 35)                1085      
                                                                 
 dense_6 (Dense)             (None, 1)                 36        
                                                                 
Total params: 2,631
Trainable params: 2,631
Non-trainable params: 0
_________________________________________________________________


In [20]:
# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [21]:
# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
804/804 [==============================] - 1s 754us/step - loss: 0.5792 - accuracy: 0.7133
Epoch 2/50
804/804 [==============================] - 1s 730us/step - loss: 0.5561 - accuracy: 0.7251
Epoch 3/50
804/804 [==============================] - 1s 697us/step - loss: 0.5533 - accuracy: 0.7292
Epoch 4/50
804/804 [==============================] - 1s 700us/step - loss: 0.5506 - accuracy: 0.7297
Epoch 5/50
804/804 [==============================] - 1s 693us/step - loss: 0.5489 - accuracy: 0.7314
Epoch 6/50
804/804 [==============================] - 1s 708us/step - loss: 0.5487 - accuracy: 0.7311
Epoch 7/50
804/804 [==============================] - 1s 702us/step - loss: 0.5480 - accuracy: 0.7309
Epoch 8/50
804/804 [==============================] - 1s 717us/step - loss: 0.5471 - accuracy: 0.7324
Epoch 9/50
804/804 [==============================] - 1s 696us/step - loss: 0.5463 - accuracy: 0.7316
Epoch 10/50
804/804 [==============================] - 1s 701us/step - loss: 0.545

In [22]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5552 - accuracy: 0.7277 - 240ms/epoch - 895us/step
Loss: 0.5551568865776062, Accuracy: 0.7276967763900757


In [23]:
# Attempt 2 Result: changing the node increased the accuracy barely from attempt 1, 
# not the right course of action

In [24]:
# Attempt 3: change units and add another layer like in attempt 2 and doubling epochs.

In [25]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE

nn_model = tf.keras.models.Sequential()

# First hidden layer
# nn_model.add(tf.keras.layers.Dense(units=15, activation="relu", input_dim=40))
nn_model.add(tf.keras.layers.Dense(units=20, activation="tanh", input_dim=43))

# Second hidden layer
# nn_model.add(tf.keras.layers.Dense(units=25, activation="relu"))
nn_model.add(tf.keras.layers.Dense(units=30, activation="tanh"))

# Third hidden layer (New Layer)
nn_model.add(tf.keras.layers.Dense(units=35, activation="tanh"))

# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 20)                880       
                                                                 
 dense_8 (Dense)             (None, 30)                630       
                                                                 
 dense_9 (Dense)             (None, 35)                1085      
                                                                 
 dense_10 (Dense)            (None, 1)                 36        
                                                                 
Total params: 2,631
Trainable params: 2,631
Non-trainable params: 0
_________________________________________________________________


In [26]:
# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [27]:
# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 779us/step - loss: 0.5744 - accuracy: 0.7178
Epoch 2/100
804/804 [==============================] - 1s 715us/step - loss: 0.5589 - accuracy: 0.7270
Epoch 3/100
804/804 [==============================] - 1s 723us/step - loss: 0.5557 - accuracy: 0.7279
Epoch 4/100
804/804 [==============================] - 1s 716us/step - loss: 0.5534 - accuracy: 0.7298
Epoch 5/100
804/804 [==============================] - 1s 742us/step - loss: 0.5520 - accuracy: 0.7277
Epoch 6/100
804/804 [==============================] - 1s 743us/step - loss: 0.5507 - accuracy: 0.7313
Epoch 7/100
804/804 [==============================] - 1s 725us/step - loss: 0.5498 - accuracy: 0.7308
Epoch 8/100
804/804 [==============================] - 1s 709us/step - loss: 0.5494 - accuracy: 0.7313
Epoch 9/100
804/804 [==============================] - 1s 734us/step - loss: 0.5484 - accuracy: 0.7310
Epoch 10/100
804/804 [==============================] - 1s 771us/step - l

804/804 [==============================] - 1s 749us/step - loss: 0.5346 - accuracy: 0.7379
Epoch 80/100
804/804 [==============================] - 1s 762us/step - loss: 0.5343 - accuracy: 0.7390
Epoch 81/100
804/804 [==============================] - 1s 704us/step - loss: 0.5344 - accuracy: 0.7395
Epoch 82/100
804/804 [==============================] - 1s 715us/step - loss: 0.5347 - accuracy: 0.7394
Epoch 83/100
804/804 [==============================] - 1s 705us/step - loss: 0.5344 - accuracy: 0.7388
Epoch 84/100
804/804 [==============================] - 1s 694us/step - loss: 0.5344 - accuracy: 0.7394
Epoch 85/100
804/804 [==============================] - 1s 696us/step - loss: 0.5344 - accuracy: 0.7395
Epoch 86/100
804/804 [==============================] - 1s 716us/step - loss: 0.5343 - accuracy: 0.7393
Epoch 87/100
804/804 [==============================] - 1s 706us/step - loss: 0.5339 - accuracy: 0.7386
Epoch 88/100
804/804 [==============================] - 1s 714us/step - loss:

In [28]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5546 - accuracy: 0.7266 - 217ms/epoch - 811us/step
Loss: 0.5546182990074158, Accuracy: 0.7266472578048706


In [29]:
# Attempt 3 Result: changing the node continued to decrease accuracy from attempt 1 AND 2.
# Optimization attempt 1 had the highest accuracy during optimization process but still 
# ultimately lower than AlphabetSoupCharity before optimization.

In [30]:
# Export our model to HDF5 file
nn_model.save('AlphabetSoupCharity-Optimization.h5')